In [10]:
import pymongo
client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
db = client.rawData
db.collection_names()
collection = client.rawData.ipeenInfo



In [17]:
import json
import pymongo
from pymongo import UpdateOne
import numpy
import math
from math import radians, cos, sin, asin, sqrt
from collections import Counter
import time
import sys
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage
import smtplib
import gc
gc.enable() 



def haversine(lng1, lat1, lng2, lat2):
    # 将十进制度数转化为弧度
    lng1, lat1, lng2, lat2 = map(radians, [lng1, lat1, lng2, lat2])
    # haversine公式
    dlng = lng2 - lng1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlng / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # 地球平均半径，单位为公里
    return c * r * 1000


b=time.time()
def mailTo(title,mailAdds,message,whoSend='taiwanInfoSuFood'):
    msg = MIMEMultipart()    
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text=msg.as_string()
    #print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1',25)
    s.sendmail(sender,mailAdds, text)
    s.quit()  

mailTo(title="taiwanInfoSuFoodbegin",mailAdds=["andy.yuan@wowprime.com"],message="分析開始")
try:
    ######################################################第一次跑用stoneTwo來篩選沒超商的點
    b=time.time()
    client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
#     collectionTaiwan = client.rawData.taiwanInfoMaBoss
    collectionTaiwan = client.rawData.taiwanInfoSuFood
    ##################################################第一次跑不執行
    TWLocs=list(collectionTaiwan.find({}, {'_id': False}))
    ##################################################第一次跑不執行
    
    ######################################################IPEEN
    # collection = client.rawData.wowprimeipeen
    collection = client.rawData.ipeenInfo
    ipeendata = list(collection.find({}, {'_id': False}))
    ipeendata = [dien for dien in ipeendata if dien['status'] == "正常營業"
                 and dien['lat'] > 18
                 and dien['lat'] < 27
                 and dien['lng'] < 125
                 and dien['lng'] > 117
                 and dien['bigadd'] != 0
                 and dien['smalladd'] != 0]

    ######################################################IPEEN
    ######################################################conStore0320
    collection = client.rawData.conStore
    conStoreData = list(collection.find({}, {'_id': False}))
    ######################################################conStore0320
    ######################################################Watsons
    collection = client.rawData.Watsons
    watsonsData = list(collection.find({}, {'_id': False}))
    ######################################################Watsons
    ######################################################canSuMe
    collection = client.rawData.canSuMe
    canSuMeData = list(collection.find({}, {'_id': False}))
    ######################################################canSuMe
#     ######################################################carrefour0504
    collection = client.rawData.MRTinfo
    MRTData = list(collection.find({}, {'_id': False}))
#     ######################################################carrefour0504
    ######################################################pxmart0504
    collection = client.rawData.eslite
    esliteData = list(collection.find({}, {'_id': False}))
    ######################################################pxmart0504
#     ######################################################Clinic0531
#     collection = client.rawData.infoClinic
#     clinicData = list(collection.find({}, {'_id': False}))
#     ######################################################Clinic0531
#     ######################################################Tstore0531
#     collection = client.rawData.info3Store
#     tStoreData = list(collection.find({}, {'_id': False}))
#     ######################################################Tstore0531
    ##################################################第一次跑先去除latlng以外的資訊
    TWLocs = [{"lat":i["lat"],"lng":i["lng"]} for i in TWLocs if ('NconStore_Analyze' in i and
              i['NconStore_Analyze'] >0)]
#     TWLocs = [i for i in TWLocs if ('NconStore_Analyze' in i and
#               i['NconStore_Analyze'] >0)]
    ##################################################第一次跑先去除latlng以外的資訊
    allN=len(TWLocs)
    print("有{}筆資料要分析".format(allN))
    radius=500
    n = 0

    for TWLoc in TWLocs:
        score=0
        n += 1
        if n % 50 == 0:
            e=time.time()
            print("已完成{}%分析-{}項,花費{}秒".format(round(n/allN,4)*100,n,round(e-b)))
        if n % 2000 == 0:
            gc.collect()
            client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
            collection = client.rawData.taiwanInfoSuFood
            
            ids=["{}_{}".format(i['lat'],i['lng']) for i in TWLocs if i.get("NconStore_Analyze",0)>0]
            datas=[i for i in TWLocs if i.get("NconStore_Analyze",0)>0]
            operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,datas)]
            try:
                collection.bulk_write(operations ,ordered=False)
            except Exception as er:
                print(er)
            client.close()

        TWLoc["areaRadius_Analyze"] = radius
        #############################################################0523Score
        #+2
        TWLoc["NconStore_Analyze"] = len([dien['name'] for dien in conStoreData if "lng" in dien and haversine(lng1=float(dien["lng"]),
                                                                                lat1=float(dien["lat"]),
                                                                                lng2=TWLoc["lng"],
                                                                                lat2=TWLoc["lat"]) <= radius])
            
        #+2
        TWLoc["Nwatson_Analyze"] = len([dien for dien in watsonsData
                                          if haversine(lng1=float(dien["lng"]),
                                                       lat1=float(dien["lat"]),
                                                       lng2=TWLoc["lng"],
                                                       lat2=TWLoc["lat"]) <= radius])
        TWLoc["NcanSuMe_Analyze"] = len([dien for dien in canSuMeData
                                          if haversine(lng1=float(dien["lng"]),
                                                       lat1=float(dien["lat"]),
                                                       lng2=TWLoc["lng"],
                                                       lat2=TWLoc["lat"]) <= radius])
        
        #+2
        TWLoc["NMRT_Analyze"] = len([dien for dien in MRTData
                                  if haversine(lng1=float(dien["lng"]),
                                               lat1=float(dien["lat"]),
                                               lng2=TWLoc["lng"],
                                               lat2=TWLoc["lat"]) <= 200])
        #+2
        TWLoc["Neslite_Analyze"] = len([dien for dien in esliteData
                                  if haversine(lng1=float(dien["lng"]),
                                               lat1=float(dien["lat"]),
                                               lng2=TWLoc["lng"],
                                               lat2=TWLoc["lat"]) <= radius])

        rangeCost = 1.4
        
        #+2
        goodDien = [dien['smallstyle'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                       lat1=dien["lat"],
                                                                       lng2=TWLoc["lng"],
                                                                       lat2=TWLoc["lat"]) <= 500
                   and dien['averagecost'] > 399]
        
        TWLoc["NgoodDien"] = len(goodDien)
        
        #-1
        Veget = len([dien['smallstyle'] for dien in ipeendata if haversine(lng1=dien["lng"],
                                                                       lat1=dien["lat"],
                                                                       lng2=TWLoc["lng"],
                                                                       lat2=TWLoc["lat"]) <= 500
                   and dien['bigstyle']=='素食'
                   and dien['smallstyle']=='西式'
                   and dien['averagecost'] < 450
                   and dien['averagecost'] > 200])
        TWLoc["NVeget_analyze"] = Veget
        

        #############################################################0523Score
    for i in TWLocs:
        score=0

        if i['NMRT_Analyze']>=1:
            score+=2

        if i['NVeget_analyze']>=1:
            score-=1

        if i['NcanSuMe_Analyze']+i["Nwatson_Analyze"]>=2:
            score+=2
        elif i['NcanSuMe_Analyze']+i["Nwatson_Analyze"]>=1:
            score+=1

        if i['NconStore_Analyze']>=13:
            score+=2
        elif i['NconStore_Analyze']>=5:
            score+=1

        if i['Neslite_Analyze']>=1:
            score+=2

        if i['NgoodDien']>=30:
            score+=2
        elif i['NgoodDien']>=15:
            score+=1

        i['score']=score
        
    
    e = time.time()


#     client.close()
    client = pymongo.mongo_client.MongoClient("192.168.1.113", 27017, username='j122085', password='850605')
    collection = client.rawData.taiwanInfoSuFood
    ids=["{}_{}".format(i['lat'],i['lng']) for i in TWLocs]
    operations=[UpdateOne({"_id":idn},{'$set':data},upsert=True) for idn ,data in zip(ids,TWLocs)]
    try:
        collection.bulk_write(operations ,ordered=False)
    except Exception as er:
        print(er)
    client.close()
    
    e = time.time()
    print(e-b)
    
    
    mailTo(title="taiwanInfoSuFoodSuccess",mailAdds=["andy.yuan@wowprime.com"],message="分析完畢_耗時{}秒".format(e-b))
except Exception as er:
    errorMsg=str(er)+"出錯位置:"+str(sys.exc_info()[2].tb_lineno)
    mailTo(title="taiwanInfoSuFoodFail",mailAdds=["andy.yuan@wowprime.com"],message=errorMsg)

有50671筆資料要分析
已完成0.1%分析-50項,花費50秒
已完成0.2%分析-100項,花費98秒
已完成0.3%分析-150項,花費149秒
已完成0.38999999999999996%分析-200項,花費202秒
已完成0.49%分析-250項,花費256秒
已完成0.59%分析-300項,花費305秒
已完成0.69%分析-350項,花費352秒
已完成0.79%分析-400項,花費398秒
已完成0.89%分析-450項,花費447秒
已完成0.9900000000000001%分析-500項,花費497秒
已完成1.09%分析-550項,花費548秒
已完成1.18%分析-600項,花費598秒
已完成1.28%分析-650項,花費645秒
已完成1.38%分析-700項,花費690秒
已完成1.48%分析-750項,花費736秒
已完成1.58%分析-800項,花費782秒
已完成1.68%分析-850項,花費828秒
已完成1.78%分析-900項,花費874秒
已完成1.87%分析-950項,花費920秒
已完成1.97%分析-1000項,花費966秒
已完成2.07%分析-1050項,花費1011秒
已完成2.17%分析-1100項,花費1058秒
已完成2.27%分析-1150項,花費1104秒
已完成2.37%分析-1200項,花費1150秒
已完成2.4699999999999998%分析-1250項,花費1197秒
已完成2.5700000000000003%分析-1300項,花費1243秒
已完成2.6599999999999997%分析-1350項,花費1288秒
已完成2.76%分析-1400項,花費1334秒
已完成2.86%分析-1450項,花費1380秒
已完成2.96%分析-1500項,花費1425秒
已完成3.06%分析-1550項,花費1470秒
已完成3.16%分析-1600項,花費1516秒
已完成3.26%分析-1650項,花費1561秒
已完成3.35%分析-1700項,花費1606秒
已完成3.45%分析-1750項,花費1652秒
已完成3.55%分析-1800項,花費1699秒
已完成3.65%分析-1850項,花費1745秒
已完成3.75%分析-1900項,花費1791秒
已完成3.85%分析-

已完成28.02%分析-14200項,花費13316秒
已完成28.12%分析-14250項,花費13362秒
已完成28.22%分析-14300項,花費13407秒
已完成28.32%分析-14350項,花費13452秒
已完成28.42%分析-14400項,花費13497秒
已完成28.52%分析-14450項,花費13542秒
已完成28.62%分析-14500項,花費13587秒
已完成28.71%分析-14550項,花費13632秒
已完成28.810000000000002%分析-14600項,花費13677秒
已完成28.910000000000004%分析-14650項,花費13722秒
已完成29.01%分析-14700項,花費13768秒
已完成29.110000000000003%分析-14750項,花費13813秒
已完成29.21%分析-14800項,花費13858秒
已完成29.310000000000002%分析-14850項,花費13903秒
已完成29.409999999999997%分析-14900項,花費13949秒
已完成29.5%分析-14950項,花費13994秒
已完成29.599999999999998%分析-15000項,花費14039秒
已完成29.7%分析-15050項,花費14085秒
已完成29.799999999999997%分析-15100項,花費14131秒
已完成29.9%分析-15150項,花費14177秒
已完成30.0%分析-15200項,花費14222秒
已完成30.099999999999998%分析-15250項,花費14267秒
已完成30.19%分析-15300項,花費14313秒
已完成30.29%分析-15350項,花費14360秒
已完成30.39%分析-15400項,花費14405秒
已完成30.490000000000002%分析-15450項,花費14450秒
已完成30.59%分析-15500項,花費14495秒
已完成30.69%分析-15550項,花費14540秒
已完成30.79%分析-15600項,花費14586秒
已完成30.89%分析-15650項,花費14631秒
已完成30.98%分析-15700項,花費14676秒
已完成31.0800000000000

已完成54.06999999999999%分析-27400項,花費25390秒
已完成54.169999999999995%分析-27450項,花費25439秒
已完成54.269999999999996%分析-27500項,花費25489秒
已完成54.37%分析-27550項,花費25537秒
已完成54.47%分析-27600項,花費25588秒
已完成54.56999999999999%分析-27650項,花費25636秒
已完成54.669999999999995%分析-27700項,花費25689秒
已完成54.769999999999996%分析-27750項,花費25740秒
已完成54.86%分析-27800項,花費25789秒
已完成54.96%分析-27850項,花費25836秒
已完成55.059999999999995%分析-27900項,花費25885秒
已完成55.16%分析-27950項,花費25931秒
已完成55.26%分析-28000項,花費25980秒
已完成55.36%分析-28050項,花費26036秒
已完成55.46%分析-28100項,花費26086秒
已完成55.55%分析-28150項,花費26133秒
已完成55.65%分析-28200項,花費26183秒
已完成55.75%分析-28250項,花費26232秒
已完成55.85%分析-28300項,花費26279秒
已完成55.95%分析-28350項,花費26325秒
已完成56.05%分析-28400項,花費26370秒
已完成56.15%分析-28450項,花費26415秒
已完成56.25%分析-28500項,花費26460秒
已完成56.34%分析-28550項,花費26506秒
已完成56.44%分析-28600項,花費26551秒
已完成56.54%分析-28650項,花費26596秒
已完成56.64%分析-28700項,花費26642秒
已完成56.74%分析-28750項,花費26687秒
已完成56.84%分析-28800項,花費26733秒
已完成56.940000000000005%分析-28850項,花費26778秒
已完成57.03%分析-28900項,花費26823秒
已完成57.13%分析-28950項,花費26868秒
已完

已完成79.93%分析-40500項,花費37134秒
已完成80.03%分析-40550項,花費37177秒
已完成80.12%分析-40600項,花費37221秒
已完成80.22%分析-40650項,花費37264秒
已完成80.32000000000001%分析-40700項,花費37308秒
已完成80.42%分析-40750項,花費37352秒
已完成80.52%分析-40800項,花費37395秒
已完成80.62%分析-40850項,花費37439秒
已完成80.72%分析-40900項,花費37482秒
已完成80.82000000000001%分析-40950項,花費37526秒
已完成80.91000000000001%分析-41000項,花費37569秒
已完成81.01%分析-41050項,花費37613秒
已完成81.11%分析-41100項,花費37657秒
已完成81.21000000000001%分析-41150項,花費37701秒
已完成81.31%分析-41200項,花費37744秒
已完成81.41000000000001%分析-41250項,花費37788秒
已完成81.51%分析-41300項,花費37832秒
已完成81.6%分析-41350項,花費37875秒
已完成81.69999999999999%分析-41400項,花費37919秒
已完成81.8%分析-41450項,花費37962秒
已完成81.89999999999999%分析-41500項,花費38005秒
已完成82.0%分析-41550項,花費38049秒
已完成82.1%分析-41600項,花費38092秒
已完成82.19999999999999%分析-41650項,花費38136秒
已完成82.3%分析-41700項,花費38179秒
已完成82.39%分析-41750項,花費38222秒
已完成82.49%分析-41800項,花費38265秒
已完成82.59%分析-41850項,花費38308秒
已完成82.69%分析-41900項,花費38350秒
已完成82.78999999999999%分析-41950項,花費38393秒
已完成82.89%分析-42000項,花費38436秒
已完成82.99%分析-42050項,花費38484秒
已

In [37]:
import pandas

df=pandas.DataFrame(TWLocs)

writer=pandas.ExcelWriter(r"D:\outputXLS\suFood2.xlsx")

sa=df.to_excel(writer)

writer.save()